<a href="https://colab.research.google.com/github/gandalf1819/SF-Opioid-Crisis/blob/master/SF_drug_CatBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
!pip install catboost

In [0]:
import sys,os
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.base import clone
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import log_loss
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
import catboost
import gensim
import seaborn as sns
sns.set()

In [4]:
d_crime = pd.read_csv("/content/gdrive/My Drive/SF dataset/Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv")
d_crime.columns

Index(['IncidntNum', 'Category', 'Descript', 'DayOfWeek', 'Date', 'Time',
       'PdDistrict', 'Resolution', 'Address', 'X', 'Y', 'Location', 'PdId'],
      dtype='object')

In [0]:
np.random.seed(100)
random_d_crime=d_crime.sample(2215024)
train_size=int(0.67 *2215024)
X_train=random_d_crime[:train_size]
X_test=random_d_crime[train_size:]

In [6]:
X_train['DateTime'] = X_train.Date + " " + X_train.Time
X_train['DateTime'] = pd.to_datetime(X_train['DateTime'], format='%m/%d/%Y %H:%M')

X_test['DateTime'] = X_test.Date + " " + X_test.Time
X_test['DateTime'] = pd.to_datetime(X_test['DateTime'], format='%m/%d/%Y %H:%M')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [7]:
X_train[:5]

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId,DateTime
1618683,61097743,SUSPICIOUS OCC,SUSPICIOUS OCCURRENCE,Friday,10/13/2006,23:13,CENTRAL,NONE,200 Block of COLUMBUS AV,-122.406504,37.797678,POINT (-122.406503626072 37.7976778969458),6109774364070,2006-10-13 23:13:00
1303833,120517541,LARCENY/THEFT,GRAND THEFT PICKPOCKET,Saturday,06/30/2012,17:40,MISSION,NONE,16TH ST / FOLSOM ST,-122.415308,37.765316,POINT (-122.415308388816 37.765316135635),12051754106111,2012-06-30 17:40:00
38943,146134967,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Sunday,07/06/2014,22:20,CENTRAL,NONE,NORTHPOINT ST / POLK ST,-122.423658,37.805285,POINT (-122.423658369809 37.8052850926566),14613496706244,2014-07-06 22:20:00
1737093,81124398,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Saturday,10/18/2008,10:30,BAYVIEW,NONE,IRWIN ST / 8TH ST,-122.399679,37.766695,POINT (-122.399679247863 37.7666952934378),8112439806244,2008-10-18 10:30:00
1490803,100564255,ASSAULT,STALKING,Saturday,04/10/2010,17:30,TENDERLOIN,NONE,500 Block of LEAVENWORTH ST,-122.414645,37.786069,POINT (-122.414645409901 37.7860688364633),10056425515201,2010-04-10 17:30:00


In [8]:
print(X_train.duplicated().sum())
X_train.drop_duplicates(inplace=True)
assert(X_train.duplicated().sum() == 0)

0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [9]:
X_test[:5]

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId,DateTime
1041060,100505055,LARCENY/THEFT,GRAND THEFT FROM PERSON,Monday,05/31/2010,13:30,NORTHERN,NONE,VANNESS AV / BUSH ST,-122.421949,37.788488,POINT (-122.421949487547 37.7884881521135),10050505506153,2010-05-31 13:30:00
578101,51217525,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Friday,10/14/2005,13:00,MISSION,NONE,2600 Block of MISSION ST,-122.418600,37.754626,POINT (-122.41860030589 37.7546255872838),5121752506244,2005-10-14 13:00:00
324260,81013597,PROSTITUTION,PIMPING,Wednesday,09/24/2008,01:39,CENTRAL,"ARREST, CITED",LARKIN ST / SUTTER ST,-122.418468,37.787965,POINT (-122.418467747931 37.7879652569797),8101359713030,2008-09-24 01:39:00
1404604,31205829,VEHICLE THEFT,STOLEN AUTOMOBILE,Sunday,10/12/2003,23:00,INGLESIDE,NONE,900 Block of HAMILTON ST,-122.406894,37.719578,POINT (-122.406893832735 37.7195778124637),3120582907021,2003-10-12 23:00:00
1357958,90022202,NON-CRIMINAL,TARASOFF REPORT,Wednesday,01/07/2009,10:42,PARK,NONE,2200 Block of GEARY BL,-122.440341,37.783324,POINT (-122.440341074545 37.7833242481047),9002220215095,2009-01-07 10:42:00


In [10]:
y_train = X_train['Category']
X_train_description = X_train['Descript']
X_train_resolution = X_train['Resolution']
X_train.drop(["Category", "Descript", "Resolution"], axis=1, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [11]:
X_test['Id'] = X_test.index

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [12]:
test_ID = X_test["Id"]
X_test.drop("Id", axis=1, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [13]:
X_train[:5]

,IncidntNum,DayOfWeek,Date,Time,PdDistrict,Address,X,Y,Location,PdId,DateTime
1618683,61097743,Friday,10/13/2006,23:13,CENTRAL,200 Block of COLUMBUS AV,-122.406504,37.797678,POINT (-122.406503626072 37.7976778969458),6109774364070,2006-10-13 23:13:00
1303833,120517541,Saturday,06/30/2012,17:40,MISSION,16TH ST / FOLSOM ST,-122.415308,37.765316,POINT (-122.415308388816 37.765316135635),12051754106111,2012-06-30 17:40:00
38943,146134967,Sunday,07/06/2014,22:20,CENTRAL,NORTHPOINT ST / POLK ST,-122.423658,37.805285,POINT (-122.423658369809 37.8052850926566),14613496706244,2014-07-06 22:20:00
1737093,81124398,Saturday,10/18/2008,10:30,BAYVIEW,IRWIN ST / 8TH ST,-122.399679,37.766695,POINT (-122.399679247863 37.7666952934378),8112439806244,2008-10-18 10:30:00
1490803,100564255,Saturday,04/10/2010,17:30,TENDERLOIN,500 Block of LEAVENWORTH ST,-122.414645,37.786069,POINT (-122.414645409901 37.7860688364633),10056425515201,2010-04-10 17:30:00


In [14]:
X_test[:5]

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId,DateTime
1041060,100505055,LARCENY/THEFT,GRAND THEFT FROM PERSON,Monday,05/31/2010,13:30,NORTHERN,NONE,VANNESS AV / BUSH ST,-122.421949,37.788488,POINT (-122.421949487547 37.7884881521135),10050505506153,2010-05-31 13:30:00
578101,51217525,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Friday,10/14/2005,13:00,MISSION,NONE,2600 Block of MISSION ST,-122.418600,37.754626,POINT (-122.41860030589 37.7546255872838),5121752506244,2005-10-14 13:00:00
324260,81013597,PROSTITUTION,PIMPING,Wednesday,09/24/2008,01:39,CENTRAL,"ARREST, CITED",LARKIN ST / SUTTER ST,-122.418468,37.787965,POINT (-122.418467747931 37.7879652569797),8101359713030,2008-09-24 01:39:00
1404604,31205829,VEHICLE THEFT,STOLEN AUTOMOBILE,Sunday,10/12/2003,23:00,INGLESIDE,NONE,900 Block of HAMILTON ST,-122.406894,37.719578,POINT (-122.406893832735 37.7195778124637),3120582907021,2003-10-12 23:00:00
1357958,90022202,NON-CRIMINAL,TARASOFF REPORT,Wednesday,01/07/2009,10:42,PARK,NONE,2200 Block of GEARY BL,-122.440341,37.783324,POINT (-122.440341074545 37.7833242481047),9002220215095,2009-01-07 10:42:00


In [15]:
X_train.shape

(1484066, 11)

In [16]:
X_test.shape

(730958, 14)

In [17]:
y_train.value_counts()

LARCENY/THEFT                  322179
OTHER OFFENSES                 207433
NON-CRIMINAL                   159533
ASSAULT                        130381
VEHICLE THEFT                   84732
DRUG/NARCOTIC                   79817
VANDALISM                       77918
WARRANTS                        68048
BURGLARY                        61295
SUSPICIOUS OCC                  53843
MISSING PERSON                  43592
ROBBERY                         37321
FRAUD                           27661
SECONDARY CODES                 17358
FORGERY/COUNTERFEITING          15487
WEAPON LAWS                     14917
TRESPASS                        13045
PROSTITUTION                    11350
STOLEN PROPERTY                  7892
SEX OFFENSES, FORCIBLE           7889
DISORDERLY CONDUCT               6794
DRUNKENNESS                      6552
RECOVERED VEHICLE                5906
DRIVING UNDER THE INFLUENCE      3760
KIDNAPPING                       3577
RUNAWAY                          2941
LIQUOR LAWS 

In [18]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)
print(le.classes_)

['ARSON' 'ASSAULT' 'BAD CHECKS' 'BRIBERY' 'BURGLARY' 'DISORDERLY CONDUCT'
 'DRIVING UNDER THE INFLUENCE' 'DRUG/NARCOTIC' 'DRUNKENNESS'
 'EMBEZZLEMENT' 'EXTORTION' 'FAMILY OFFENSES' 'FORGERY/COUNTERFEITING'
 'FRAUD' 'GAMBLING' 'KIDNAPPING' 'LARCENY/THEFT' 'LIQUOR LAWS' 'LOITERING'
 'MISSING PERSON' 'NON-CRIMINAL' 'OTHER OFFENSES'
 'PORNOGRAPHY/OBSCENE MAT' 'PROSTITUTION' 'RECOVERED VEHICLE' 'ROBBERY'
 'RUNAWAY' 'SECONDARY CODES' 'SEX OFFENSES, FORCIBLE'
 'SEX OFFENSES, NON FORCIBLE' 'STOLEN PROPERTY' 'SUICIDE' 'SUSPICIOUS OCC'
 'TREA' 'TRESPASS' 'VANDALISM' 'VEHICLE THEFT' 'WARRANTS' 'WEAPON LAWS']


In [19]:
num_train = X_train.shape[0]
all_data = pd.concat((X_train, X_test), ignore_index=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [0]:
date = pd.to_datetime(all_data['DateTime'])
all_data['year'] = date.dt.year
all_data['month'] = date.dt.month
all_data['day'] = date.dt.day
all_data['hour'] = date.dt.hour
all_data['minute'] = date.dt.minute
all_data['special_time'] = all_data['minute'].isin([0, 30]).astype(int)
# all_data['second'] = date.dt.second  # all zero
all_data["n_days"] = (date - date.min()).apply(lambda x: x.days)
all_data.drop("DateTime", axis=1, inplace=True)

In [21]:
all_data["DayOfWeek"].value_counts()

Friday       337941
Wednesday    324020
Saturday     320593
Thursday     316156
Tuesday      315135
Monday       306587
Sunday       294592
Name: DayOfWeek, dtype: int64

In [22]:
all_data["PdDistrict"].value_counts()

SOUTHERN      399785
MISSION       300076
NORTHERN      272713
CENTRAL       226255
BAYVIEW       221000
INGLESIDE     194180
TENDERLOIN    191746
TARAVAL       166971
PARK          125479
RICHMOND      116818
Name: PdDistrict, dtype: int64

In [0]:
sentences = []
for s in all_data["Address"]:
    sentences.append(s.split(" "))
address_model = gensim.models.Word2Vec(sentences, min_count=1)
encoded_address = np.zeros((all_data.shape[0], 100))
for i in range(len(sentences)):
    for j in range(len(sentences[i])):
        encoded_address[i] += address_model.wv[sentences[i][j]]
    encoded_address[i] /= len(sentences[i])

In [0]:
all_data['block'] = all_data["Address"].str.contains("block", case=False)
all_data.drop("Address", axis=1, inplace=True)

In [0]:
X_median = all_data[all_data["X"] < -120.5]["X"].median()
Y_median = all_data[all_data["Y"] < 90]["Y"].median()
all_data.loc[all_data["X"] >= -120.5, "X"] = X_median
all_data.loc[all_data["Y"] >= 90, "Y"] = Y_median

In [26]:
print(all_data["X"].min(), all_data["X"].max())
print(all_data["Y"].min(), all_data["Y"].max())

-122.51364206429 -122.364750704393
37.7078790224135 37.820620838070205


In [27]:
all_data["X+Y"] = all_data["X"] + all_data["Y"]
all_data["X-Y"] = all_data["X"] - all_data["Y"]
all_data["XY30_1"] = all_data["X"] * np.cos(np.pi / 6) + all_data["Y"] * np.sin(np.pi / 6)
all_data["XY30_2"] = all_data["Y"] * np.cos(np.pi / 6) - all_data["X"] * np.sin(np.pi / 6)
all_data["XY60_1"] = all_data["X"] * np.cos(np.pi / 3) + all_data["Y"] * np.sin(np.pi / 3)
all_data["XY60_2"] = all_data["Y"] * np.cos(np.pi / 3) - all_data["X"] * np.sin(np.pi / 3)
all_data["XY1"] = (all_data["X"] - all_data["X"].min()) ** 2 + (all_data["Y"] - all_data["Y"].min()) ** 2
all_data["XY2"] = (all_data["X"].max() - all_data["X"]) ** 2 + (all_data["Y"] - all_data["Y"].min()) ** 2
all_data["XY3"] = (all_data["X"] - all_data["X"].min()) ** 2 + (all_data["Y"].max() - all_data["Y"]) ** 2
all_data["XY4"] = (all_data["X"].max() - all_data["X"]) ** 2 + (all_data["Y"].max() - all_data["Y"]) ** 2
all_data["XY5"] = (all_data["X"] - X_median) ** 2 + (all_data["Y"] - Y_median) ** 2
pca = PCA(n_components=2).fit(all_data[["X", "Y"]])
XYt = pca.transform(all_data[["X", "Y"]])
all_data["XYpca1"] = XYt[:, 0]
all_data["XYpca2"] = XYt[:, 1]
# n_components selected by aic/bic
clf = GaussianMixture(n_components=150, covariance_type="diag",
                      random_state=0).fit(all_data[["X", "Y"]])
all_data["XYcluster"] = clf.predict(all_data[["X", "Y"]])

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-27-669610a4cb46>", line 18, in <module>
    random_state=0).fit(all_data[["X", "Y"]])
  File "/usr/local/lib/python3.6/dist-packages/sklearn/mixture/base.py", line 191, in fit
    self.fit_predict(X, y)
  File "/usr/local/lib/python3.6/dist-packages/sklearn/mixture/base.py", line 241, in fit_predict
    log_prob_norm, log_resp = self._e_step(X)
  File "/usr/local/lib/python3.6/dist-packages/sklearn/mixture/base.py", line 294, in _e_step
    log_prob_norm, log_resp = self._estimate_log_prob_resp(X)
  File "/usr/local/lib/python3.6/dist-packages/sklearn/mixture/base.py", line 503, in _estimate_log_prob_resp
    weighted_log_prob = self._estimate_weighted_log_prob(X)
  File "/usr/local/lib/python3.6/dist-packages/sklearn/mixture/base.py", line 456, in _estimate_weighted_log_p

KeyboardInterrupt: ignored

In [0]:
all_data.fillna(all_data.mean())

In [0]:
categorical_features = ["DayOfWeek", "PdDistrict", "block", "special_time", "XYcluster"]
ct = ColumnTransformer(transformers=[("categorical_features", OrdinalEncoder(), categorical_features)],
                       remainder="passthrough")
all_data = ct.fit_transform(all_data)